In [2]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 

class UpworkAnalyzer:
    
    def __init__(self):
        # Settings
        self.present_date = "10/2017" # This is the month in which the data was collected
        self.all_data_file_name = 'trial_11_9_2017_upwork_analysis_trial.csv' # Filename for all data
        self.gender_data_file_name = '11_20_2017_upwork_gender_analysis_worldwide_allskills.csv' # Filename for gender data
        self.data_log_file_name = 'trial_log_upwork_data_analysis_2017_11_17_worldwide_allskills.txt'
        self.gender_log_file_name = "log_upwork_identify_gender_2017_11_20_worldwide_allskills.txt"
        self.github_photo_path = 'https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/resized_profile_photos/resized_images_worldwide_allskills_2017_10_21/'
        
        # Write a log
        self.log = open(self.data_log_file_name, 'a')
        self.photo_log = open(self.gender_log_file_name, 'a')
        self.log.write("We have started analyzing data!" + "\n")
        self.log.flush()
        self.photo_log.write("We have started identifying gender in photos!" + "\n")
        self.photo_log.flush()

        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Get detailed_info from workers in our database
        self.cur.execute("SELECT detailed_info FROM upwork_worldwide_allskills_2017_10_21;")
        
        # Initialize arrays for Causal Analysis 
        self.user_count = 1
        self.bill_rate_array = []
        self.gender_array = []
        self.all_covariates_array = []
        self.country_id_list = []
        self.job_category_id_list = []
    
    def save_all_to_csv(self):
        with open(self.all_data_file_name, 'w') as csvfile:
            fieldnames = ['user_count','worker_id', 'first_name', 'bill_rate', 
                          'country', 'country_id', 'degree', 'education', 
                          'work_experience', 'job_category', 'job_category_id']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    first_name = user[0]["dev_first_name"].encode('utf-8').strip()
                    bill_rate = user[0]["dev_bill_rate"]
                    country = user[0]["dev_country"]
                    country_id = self.transform_country_id(country)
                    degree = self.show_degree(user)
                    education = self.calculate_education(user)
                    work_experience = self.calculate_work_experience(user)
                    job_category = self.identify_job_category(user)
                    job_category_id = self.transform_job_category(job_category)
            
                    writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'first_name': first_name,
                                 'bill_rate': bill_rate, 'country': country, 'country_id': country_id, 'degree': degree, 'education': education, 
                                 'work_experience': work_experience, 'job_category': job_category, 'job_category_id': job_category_id})
                except:
                    print "Ran into some error at user {0}".format(self.user_count)
                    print json.dumps(user[0], indent=2)
                    
                self.user_count += 1
    
    def save_gender_to_csv(self): # Identify gender of users in photos that have already been pushed to GitHub 
        with open(self.gender_data_file_name, 'w') as csvfile:
            fieldnames = ['user_count','worker_id', 'gender']
            gender_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            gender_writer.writeheader()
            
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    gender = self.identify_gender()
     
                    gender_writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'gender': gender})
        
                except Exception as err:
                    user_count = self.user_count
                    print "Ran into some error saving gender to csv at {0}".format(self.user_count)
                    self.photo_log.write("Ran into an error saving gender to csv at {0}: {1}".format(self.user_count, err) + "\n")
                    self.photo_log.flush()
                    gender_writer.writerow({'user_count': user_count, 'worker_id': "error", 'gender': "error"})
                    
                self.user_count += 1
        
    def identify_gender(self): # Returns gender as a string
        if (self.user_count%10 == 0): # Set timeout for rate limiting; paid subscription up to 10 per second 
            time.sleep(2)
        
        try: 
            print "Recognizing Face Number " + str(self.user_count)
            raw_face_data = self.recognize_faces()
            
            if(len(raw_face_data) == 0): 
                return "unidentified"
            
            else:
                gender = raw_face_data[0]["faceAttributes"]["gender"]
                return str(gender)
            
        except Exception as err:
            print err
            self.photo_log.write("Ran into an error in identifying gender at {0}: {1}".format(self.user_count, err) + "\n")
            self.photo_log.flush()
            return "error"
        
    def recognize_faces(self):
        # Replace the subscription_key string value with your valid subscription key.
        subscription_key = 'd6ef52f1ff334ace97271267626f11bc' # This is key 1 for the paid subscription 

        # Replace or verify the region.
        uri_base = 'eastus2.api.cognitive.microsoft.com'

        # Request headers.
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': subscription_key,
        }

        # Request parameters.
        params = urllib.urlencode({
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'false',
            'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
        })

        # The URL of a JPEG image to analyze.
        beg_body = "{'url':"
        end_body = "}"
        path = self.github_photo_path + str(self.user_count) + ".jpg"
        body = beg_body + '"' + path + '"' + end_body
        
        try:
            # Execute the REST API call and get the response.
            conn = httplib.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
            conn.request("POST", "/face/v1.0/detect?%s" % params, body, headers)
            response = conn.getresponse()
            data = response.read()
            parsed = json.loads(data)
            return parsed
            
        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))
            return "error"
            
        '''
        # 'data' contains the JSON data. The following formats the JSON data for display.
        parsed = json.loads(data)
        print ("Response:")
        print (json.dumps(parsed, sort_keys=True, indent=2))
        conn.close()
        '''
    
    def calculate_work_experience(self, user):
        total_experience = 0
        work_experience_array = []
        
        try: 
            work_experience_list = user[0]["experiences"]["experience"]
            
        except:
            return 0
        
        if (type(work_experience_list) is list):
            count = 0
            total_overlap = 0
            for experience in work_experience_list:
                start_date = experience["exp_from"]
                end_date = experience["exp_to"]
                if (end_date == "Present"):
                    end_date = self.present_date
                start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
                end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")
                work_experience_array.append([start_datetime, end_datetime])
                
                experience_duration = (end_datetime - start_datetime).days
                total_experience += experience_duration
            
            while (count < len(work_experience_array) - 1): # Check for overlap in work experience 
                start_newer = work_experience_array[count][0]
                end_newer = work_experience_array[count][1]
                start_older = work_experience_array[count+1][0]
                end_older = work_experience_array[count+1][1] 
                
                if (end_newer == end_older): # Pattern 1
                    newer_duration = (end_newer - start_newer).days
                    older_duration = (end_older - start_older).days
                    overlap = min(newer_duration, older_duration)
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 1"
                    print overlap
                
                elif(start_older == start_newer): # Pattern 2
                    overlap = (end_older - start_older).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 2"
                    print overlap
                
                elif(start_newer < start_older and end_newer > end_older): # Pattern 3
                    overlap = (end_older - start_older).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 3"
                    print overlap
                
                elif(start_newer > start_older and end_newer > end_older and end_older > start_newer): # Pattern 4
                    overlap = (end_older - start_newer).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 4"
                    print overlap 
                        
                else:
                    print "No overlap"
                    count += 1
                    
            total_experience = (total_experience - total_overlap)/30
            total_experience = total_experience/12 # Convert to years
            
            return total_experience
            
        elif (type(work_experience_list) is dict):
            start_date = work_experience_list["exp_from"]
            end_date = work_experience_list["exp_to"]
            if (end_date == "Present"):
                end_date = self.present_date
            start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
            end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")          
            
            total_experience = ((end_datetime - start_datetime).days)/30
            total_experience = total_experience/12 # Convert to years
            
            return total_experience
        
        else: 
            return 0
            
    
    ''' This doesn't work and we don't need it anyway 
    def calculate_months(self, start_date, end_date):
        number_of_months = 0
        present_month = int(self.present_date[0:2])
        present_year = int(self.present_date[5:7])
        start_month = int(start_date[0:2])
        start_year = int(start_date[5:7])
        
        if (end_date == "Present"):
            if (start_year == present_year):
                number_of_months = present_month - start_month # Don't count the first month so users who changed jobs immediately don't get exp. advantage
                return number_of_months
                
            else:
                if(start_year < present_year):
                    number_of_months += 12 - start_month # Don't count the first month so users who changed jobs immediately don't get exp. advantage
                    start_year += 1
                    start_month = 1
                    while (start_year < present_year):
                        number_of_months += 13 - start_month 
                        start_year += 1
                        start_month = 1
                number_of_months += present_month
                return number_of_months
                
        else:
            end_month = int(end_date[0:2])
            end_year = int(end_date[5:7])
            
            if(start_year < end_year):
                number_of_months += 12 - start_month # Don't count the first month so users who changed jobs immediately don't get exp. advantage
                start_year += 1
                start_month = 1
                while (start_year < end_year):
                    number_of_months += 13 - start_month 
                    start_year += 1
                    start_month = 1
            number_of_months += end_month
            return number_of_months
    '''
    
    def show_degree(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "No edu listed"
            elif(type(ed_history) == dict):
                return ed_history["ed_degree"]
            elif(type(ed_history) == list):
                degree_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        degree_list.append("none")
                    elif (ed_experience["ed_degree"] != ""):
                        degree_list.append(ed_experience["ed_degree"])
                return degree_list
            else:
                return "Error: Some other error"
                
        except:
            return "Error: No education"
        
    def calculate_education(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "None"
            elif(type(ed_history) == dict):
                if (ed_history["ed_degree"] == ""):
                    highest_education = self.check_highest_education("none")
                elif (ed_history["ed_degree"] != ""):
                    highest_education = self.check_highest_education(ed_history["ed_degree"])
                
                # Check for highest level of degree
                if(highest_education == "No edu listed"): # Don't count anything we can't identify
                    return "None"
                elif(highest_education == "Other"): # Don't count anything we can't identify
                    return "None"
                elif(highest_education == "Doctorate"):
                    return "Doctorate"
                elif(highest_education == "Professional"):
                    return "Professional"
                elif(highest_education == "Master"):
                    return "Master"
                elif(highest_education == "Bachelor"):
                    return "Bachelor"
                elif(highest_education == "Associate"):
                    return "Associate"
                elif(highest_education == "High School"):
                    return "High School"
                else:
                    return "None"
      
            elif(type(ed_history) == list):
                ed_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        highest_education = self.check_highest_education("none")
                        ed_list.append(highest_education)
                    elif (ed_experience["ed_degree"] != ""):
                        highest_education = self.check_highest_education(ed_experience["ed_degree"])
                        ed_list.append(highest_education)
                # Check for highest level of degree
                if("No edu listed" in ed_list): # Don't count anything we can't identify
                    return "None"
                elif("Other" in ed_list): # Don't count anything we can't identify
                    return "None"
                elif ("Doctorate" in ed_list):
                    return "Doctorate"
                elif("Professional" in ed_list):
                    return "Professional"
                elif("Master" in ed_list):
                    return "Master"
                elif("Bachelor" in ed_list):
                    return "Bachelor"
                elif("Associate" in ed_list):
                    return "Associate"
                elif("High School" in ed_list):
                    return "High School"
                else:
                    return "None"
            else:
                return "Error: Some other error"
                
        except:
            return "None"
    
    def check_highest_education(self, degree):
        # Checks for the highest level of education attained by user (DO NOT CHANGE ORDER!) 
        
        # Professional degree
        if (re.search('d\.c|d\.c\.m|d\.d\.s|d\.m\.d|ll\.b|ll\.m|l\.l\.m|l\.l\.b|j\.d\.|m\.d\.|o\.d\.|d\.o\.|pharm\.d', degree.lower()) 
            or re.search('d\.p\.m|d\.p|pod\.d|m\.div|m\.h\.l|b\.d|ordination|d\.v\.m| law', degree.lower())):
            return "Professional"
        
        # Masters
        elif (re.search('Master|Engineer\'s degree|M\.B\.A', degree)):
            return "Master"

        # Bachelors
        elif (re.search('bachelor|b\.a\.|b\.s\.', degree.lower())):
            return "Bachelor"
        

        # Associate's Degree
        elif (re.search('associate|diploma', degree.lower())):
            return "Associate"

        # PhD
        elif (re.search('doctor', degree.lower())):
            return "Doctorate"

        # High School
        elif (re.search('high school', degree.lower())):
            return "High School"

        elif (re.search('none', degree)):
            return "No edu listed"

        else:
            return "Other" # We couldn't recognize the degree 
        
    
    def identify_job_category(self, user):
        try:
            all_job_categories = user[0]["dev_job_categories_v2"]["dev_job_categories_v"]
            
        except:
            job_category = "none"
            return job_category
        
        try:
            job_category = all_job_categories[0]["groups"]["group"]["name"] # Returns general job category
        
        except:
            job_category = all_job_categories["groups"]["group"]["name"]
        
        return job_category
    
    def transform_country_id(self, country):
        if not(country in self.country_id_list):
            self.country_id_list.append(country)
        
        country_index = self.country_id_list.index(country)
        return country_index
    
    def transform_job_category(self, job_category):
        if not(job_category in self.job_category_id_list):
            self.job_category_id_list.append(job_category)
            
        job_category_index = self.job_category_id_list.index(job_category)
        return job_category_index
    
    def causal_analysis(self):
        df = pd.read_csv(self.all_data_file_name)
        df_gender = pd.read_csv(self.gender_data_file_name)
        
        all_bill_rates = df.bill_rate
        all_work_experience = df.work_experience
        all_education = df.education
        all_job_category_id = df.job_category_id
        all_country_id = df.country_id
        all_genders = df_gender.gender
        
        # Data formatting 
        for rate in all_bill_rates:
            rate = round(rate, 2)
            self.bill_rate_array.append(rate)
            
        for gender in all_genders:
            self.gender_array.append(gender)
        
        for row in df.itertuples():
            if (row.education != "None"): # Remove users for which we don't know education 
                covariate_matrix = [row.work_experience, row.education, row.country_id, row.job_category_id]
                self.all_covariates_array.append(covariate_matrix)
            else:
                covariate_matrix = [None] # For users we don't know education of, add empty covariate matrix
                self.all_covariates_array.append(covariate_matrix)
                
        # Check that arrays contain complete data
        print "Bill rate array length: {0}".format(len(self.bill_rate_array))
        print "Gender array length: {0}".format(len(self.gender_array))
        print "Covariate array length: {0}".format(len(self.all_covariates_array))
        
        # Run causal analysis 
        Y = np.array(self.bill_rate_array)
        D = np.array(self.gender_array)
        X = np.array(self.all_covariates_array)
        causal = CausalModel(Y, D, X)
        causal.est_propensity()

        print (causal.est_propensity())
    

In [3]:
allDataObject = UpworkAnalyzer()
allDataObject.save_gender_to_csv()

Recognizing Face Number 1
Recognizing Face Number 2
Recognizing Face Number 3
Recognizing Face Number 4
Recognizing Face Number 5
Recognizing Face Number 6
Recognizing Face Number 7
Recognizing Face Number 8
Recognizing Face Number 9
Recognizing Face Number 10
Recognizing Face Number 11
Recognizing Face Number 12
Recognizing Face Number 13
Recognizing Face Number 14
Recognizing Face Number 15
Recognizing Face Number 16
Recognizing Face Number 17
Recognizing Face Number 18
Recognizing Face Number 19
Recognizing Face Number 20
Recognizing Face Number 21
Recognizing Face Number 22
Recognizing Face Number 23
Recognizing Face Number 24
Recognizing Face Number 25
Recognizing Face Number 26
Recognizing Face Number 27
Recognizing Face Number 28
Recognizing Face Number 29
Recognizing Face Number 30
Recognizing Face Number 31
Recognizing Face Number 32
Recognizing Face Number 33
Recognizing Face Number 34
Recognizing Face Number 35
Recognizing Face Number 36
Recognizing Face Number 37
Recognizin

Recognizing Face Number 295
Recognizing Face Number 296
Recognizing Face Number 297
Recognizing Face Number 298
Recognizing Face Number 299
Recognizing Face Number 300
Recognizing Face Number 301
Recognizing Face Number 302
Recognizing Face Number 303
Recognizing Face Number 304
Recognizing Face Number 305
Recognizing Face Number 306
Recognizing Face Number 307
Recognizing Face Number 308
Recognizing Face Number 309
Recognizing Face Number 310
Recognizing Face Number 311
Recognizing Face Number 312
Recognizing Face Number 313
Recognizing Face Number 314
Recognizing Face Number 315
Recognizing Face Number 316
Recognizing Face Number 317
Recognizing Face Number 318
Recognizing Face Number 319
Recognizing Face Number 320
Recognizing Face Number 321
Recognizing Face Number 322
Recognizing Face Number 323
Recognizing Face Number 324
Recognizing Face Number 325
Recognizing Face Number 326
Recognizing Face Number 327
Recognizing Face Number 328
Recognizing Face Number 329
Recognizing Face Num

Recognizing Face Number 586
Recognizing Face Number 587
Recognizing Face Number 588
Recognizing Face Number 589
Recognizing Face Number 590
Recognizing Face Number 591
Recognizing Face Number 592
Recognizing Face Number 593
Recognizing Face Number 594
Recognizing Face Number 595
Recognizing Face Number 596
Recognizing Face Number 597
Recognizing Face Number 598
Recognizing Face Number 599
Recognizing Face Number 600
Recognizing Face Number 601
Recognizing Face Number 602
Recognizing Face Number 603
Recognizing Face Number 604
Recognizing Face Number 605
Recognizing Face Number 606
Recognizing Face Number 607
Recognizing Face Number 608
Recognizing Face Number 609
Recognizing Face Number 610
Recognizing Face Number 611
Recognizing Face Number 612
Recognizing Face Number 613
Recognizing Face Number 614
Recognizing Face Number 615
Recognizing Face Number 616
Recognizing Face Number 617
Recognizing Face Number 618
Recognizing Face Number 619
Recognizing Face Number 620
Recognizing Face Num

Recognizing Face Number 877
Recognizing Face Number 878
Recognizing Face Number 879
Recognizing Face Number 880
Recognizing Face Number 881
Recognizing Face Number 882
Recognizing Face Number 883
Recognizing Face Number 884
Recognizing Face Number 885
Recognizing Face Number 886
Recognizing Face Number 887
Recognizing Face Number 888
Recognizing Face Number 889
Recognizing Face Number 890
Recognizing Face Number 891
Recognizing Face Number 892
Recognizing Face Number 893
Recognizing Face Number 894
Recognizing Face Number 895
Recognizing Face Number 896
Recognizing Face Number 897
Recognizing Face Number 898
Recognizing Face Number 899
Recognizing Face Number 900
Recognizing Face Number 901
Recognizing Face Number 902
Recognizing Face Number 903
Recognizing Face Number 904
Recognizing Face Number 905
Recognizing Face Number 906
Recognizing Face Number 907
Recognizing Face Number 908
Recognizing Face Number 909
Recognizing Face Number 910
Recognizing Face Number 911
Recognizing Face Num

Recognizing Face Number 1164
Recognizing Face Number 1165
Recognizing Face Number 1166
Recognizing Face Number 1167
Recognizing Face Number 1168
Recognizing Face Number 1169
Recognizing Face Number 1170
Recognizing Face Number 1171
Recognizing Face Number 1172
Recognizing Face Number 1173
Recognizing Face Number 1174
Recognizing Face Number 1175
Recognizing Face Number 1176
Recognizing Face Number 1177
Recognizing Face Number 1178
Recognizing Face Number 1179
Recognizing Face Number 1180
Recognizing Face Number 1181
Recognizing Face Number 1182
Recognizing Face Number 1183
Recognizing Face Number 1184
Recognizing Face Number 1185
Recognizing Face Number 1186
Recognizing Face Number 1187
Recognizing Face Number 1188
Recognizing Face Number 1189
Recognizing Face Number 1190
Recognizing Face Number 1191
Recognizing Face Number 1192
Recognizing Face Number 1193
Recognizing Face Number 1194
Recognizing Face Number 1195
Recognizing Face Number 1196
Recognizing Face Number 1197
Recognizing Fa

...

Recognizing Face Number 9647
Recognizing Face Number 9648
Recognizing Face Number 9649
Recognizing Face Number 9650
Recognizing Face Number 9651
Recognizing Face Number 9652
Recognizing Face Number 9653
Recognizing Face Number 9654
Recognizing Face Number 9655
Recognizing Face Number 9656
Recognizing Face Number 9657
Recognizing Face Number 9658
Recognizing Face Number 9659
Recognizing Face Number 9660
Recognizing Face Number 9661
Recognizing Face Number 9662
Recognizing Face Number 9663
Recognizing Face Number 9664
Recognizing Face Number 9665
Recognizing Face Number 9666
Recognizing Face Number 9667
Recognizing Face Number 9668
Recognizing Face Number 9669
Recognizing Face Number 9670
Recognizing Face Number 9671
Recognizing Face Number 9672
Recognizing Face Number 9673
Recognizing Face Number 9674
Recognizing Face Number 9675
Recognizing Face Number 9676
Recognizing Face Number 9677
Recognizing Face Number 9678
Recognizing Face Number 9679
Recognizing Face Number 9680
Recognizing Fa

Recognizing Face Number 9930
Recognizing Face Number 9931
Recognizing Face Number 9932
Recognizing Face Number 9933
Recognizing Face Number 9934
Recognizing Face Number 9935
Recognizing Face Number 9936
Recognizing Face Number 9937
Recognizing Face Number 9938
Recognizing Face Number 9939
Recognizing Face Number 9940
Recognizing Face Number 9941
Recognizing Face Number 9942
Recognizing Face Number 9943
Recognizing Face Number 9944
Recognizing Face Number 9945
Recognizing Face Number 9946
Recognizing Face Number 9947
Recognizing Face Number 9948
Recognizing Face Number 9949
Recognizing Face Number 9950
Recognizing Face Number 9951
Recognizing Face Number 9952
Recognizing Face Number 9953
Recognizing Face Number 9954
Recognizing Face Number 9955
Recognizing Face Number 9956
Recognizing Face Number 9957
Recognizing Face Number 9958
Recognizing Face Number 9959
Recognizing Face Number 9960
Recognizing Face Number 9961
Recognizing Face Number 9962
Recognizing Face Number 9963
Recognizing Fa

Recognizing Face Number 10206
Recognizing Face Number 10207
Recognizing Face Number 10208
Recognizing Face Number 10209
Recognizing Face Number 10210
Recognizing Face Number 10211
Recognizing Face Number 10212
Recognizing Face Number 10213
Recognizing Face Number 10214
Recognizing Face Number 10215
Recognizing Face Number 10216
Recognizing Face Number 10217
Recognizing Face Number 10218
Recognizing Face Number 10219
Recognizing Face Number 10220
Recognizing Face Number 10221
Recognizing Face Number 10222
Recognizing Face Number 10223
Recognizing Face Number 10224
Recognizing Face Number 10225
Recognizing Face Number 10226
Recognizing Face Number 10227
Recognizing Face Number 10228
Recognizing Face Number 10229
Recognizing Face Number 10230
Recognizing Face Number 10231
Recognizing Face Number 10232
Recognizing Face Number 10233
Recognizing Face Number 10234
Recognizing Face Number 10235
Recognizing Face Number 10236
Recognizing Face Number 10237
Recognizing Face Number 10238
Recognizin

Recognizing Face Number 10479
Recognizing Face Number 10480
Recognizing Face Number 10481
Recognizing Face Number 10482
Recognizing Face Number 10483
Recognizing Face Number 10484
Recognizing Face Number 10485
Recognizing Face Number 10486
Recognizing Face Number 10487
Recognizing Face Number 10488
Recognizing Face Number 10489
Recognizing Face Number 10490
Recognizing Face Number 10491
Recognizing Face Number 10492
Recognizing Face Number 10493
Recognizing Face Number 10494
Recognizing Face Number 10495
Recognizing Face Number 10496
Recognizing Face Number 10497
Recognizing Face Number 10498
Recognizing Face Number 10499
Recognizing Face Number 10500
Recognizing Face Number 10501
Recognizing Face Number 10502
Recognizing Face Number 10503
Recognizing Face Number 10504
Recognizing Face Number 10505
Recognizing Face Number 10506
Recognizing Face Number 10507
Recognizing Face Number 10508
Recognizing Face Number 10509
Recognizing Face Number 10510
Recognizing Face Number 10511
Recognizin

Recognizing Face Number 10753
Recognizing Face Number 10754
Recognizing Face Number 10755
Recognizing Face Number 10756
Recognizing Face Number 10757
Recognizing Face Number 10758
Recognizing Face Number 10759
Recognizing Face Number 10760
Recognizing Face Number 10761
Recognizing Face Number 10762
Recognizing Face Number 10763
Recognizing Face Number 10764
Recognizing Face Number 10765
Recognizing Face Number 10766
Recognizing Face Number 10767
Recognizing Face Number 10768
Recognizing Face Number 10769
Recognizing Face Number 10770
Recognizing Face Number 10771
Recognizing Face Number 10772
Recognizing Face Number 10773
Recognizing Face Number 10774
Recognizing Face Number 10775
Recognizing Face Number 10776
Recognizing Face Number 10777
Recognizing Face Number 10778
Recognizing Face Number 10779
Recognizing Face Number 10780
Recognizing Face Number 10781
Recognizing Face Number 10782
Recognizing Face Number 10783
Recognizing Face Number 10784
Recognizing Face Number 10785
Recognizin

In [ ]:
allDataObject.save_all_to_csv()

In [48]:
checkOverlapObject = UpworkAnalyzer()
checkOverlapObject.check_overlap("06/2017", "08/2017")
checkOverlapObject.check_overlap("06/2017", "05/2017")

Yes overlap
No overlap 


In [7]:
print "Hello world"

Hello world


In [29]:
for user in cur:
    print "Now we're on user number {0}".format(user_count)
    try:
        # Outcome - Bill Rate (Y)

        # Treatment - Gender (D)
        
        # Matrix of Covariates - Work Experience, Education, Country, Job Category (X)
        
        worker_id = user[0]["ciphertext"]
        print worker_id
        first_name = user[0]["dev_first_name"]
        print first_name
        
        gender_by_detector = "none"
        user_count += 1
        

    except:
        print "This one has no detailed info: {0}".format(user_count)
        no_detailed_info_count += 1

print bill_rate_array 

Now we're on user number 0
~019ef3758cf73d289c
Valentina
Now we're on user number 1
~014c24de20d2446d42
Marwen
Now we're on user number 2
~01f5a19ca9880b867f
Bruno
Now we're on user number 3
~0191a24ec4aadf154b
Matt
Now we're on user number 4
~019309dfcb2b02ed10
Orkun
Now we're on user number 5
~01dc5d676329574590
Ganang
Now we're on user number 6
~016e70f671e01d08f5
Yash
Now we're on user number 7
~0193675f0ca4d4c14a
Alex
Now we're on user number 8
~0125bee9e4f17ef78e
Rayhan
Now we're on user number 9
~01326a7a95a824e410
Minas


In [2]:
genderObject = UpworkAnalyzer()
genderObject.save_gender_to_csv()

CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
Master
CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
Bachelor
CHECKING HIGHEST EDUCATION
High School
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
Professional
CHECKING HIGHEST EDUCATION
None
More than one edu experience
CHECKING HIGHEST EDUCATION
Bachelor
CHECKING HIGHEST EDUCATION
Associate
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
CHECKING HIGHEST EDUCATION
More than one edu experience
CHECKING HIGHEST EDUCATION
Professional
CHECKING HIGHEST EDUCATION
Master
More than one edu experience
CHECKING

In [2]:
# Testing CausalModel 

Y = np.array([13.00, 15.00, 16.00, 14.00, 14.50, 16.50, 18.50])
D = np.array([0, 1, 0, 1, 1, 1, 0])
X = np.array([[1, 1], [2, 2], 
              [2, 1], [1, 1], 
              [2, 1], [1, 2],[2, 2]])
causal = CausalModel(Y, D, X)
causal.est_propensity()

print (causal.propensity)

ValueError: Too few control units: N_c < K+1